# Reading OccurrenceCubeWERBirds data

In [45]:

import pandas as pd
import re

# geometry:
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.wkt import loads
import geodatasets

import matplotlib.pyplot as plt



##file_path = 'https://github.com/FAIRiCUBE/data-requests/blob/main/encoding-examples/datacube_nl_farmland_birds_1.csv'
file_path =r"N:\C2205_FAIRiCUBE\f02_data\d100_species_distribution\data\datacube_nl_farmland_birds_1.csv"
file_path =r"N:\C2205_FAIRiCUBE\f02_data\d100_species_distribution\data\datacube_nl_farmland_birds_1_sub.csv"
df = pd.read_csv(file_path)
df.columns = ['year', 'gridnum_10m', 'species_id','B','C']

# Extraxt the EAST coordinate
def extract_east_number(text):
    numbers = re.findall(r'\d+', text)
    if len(numbers) >= 2:
        return int(numbers[1])  # Extract the second number
    else:
        return None

# Extraxt the NORTH coordinate
def extract_north_number(text):
    numbers = re.findall(r'\d+', text)
    if len(numbers) >= 2:
        return int(numbers[2])  # Extract the second number
    else:
        return None

# Apply the function to the text_column and create a new column
df['east_number'] = df['gridnum_10m'].apply(lambda x: extract_east_number(x))
df['north_number'] = df['gridnum_10m'].apply(lambda x: extract_north_number(x))

## get final coordinate: - for 10m reslolution:
df['east_coordinate_epsg3035_meter'] = df['east_number'] * 10
df['north_coordinate_epsg3035_meter'] = df['north_number'] * 10


print (df)
## geometry to df:
# Create geometry column using latitude and longitude

geometry = [Point(xy) for xy in zip(df['east_coordinate_epsg3035_meter'], df['north_coordinate_epsg3035_meter'])]



geometry_test = [Point(xy) for xy in zip(df['east_coordinate_epsg3035_meter'], df['north_coordinate_epsg3035_meter'])]



# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Specify the projection
gdf.crs = "EPSG:3035" 




gs_buffer = gdf.buffer(100)


## gdf.plot(marker='o', color='blue', markersize=5)
## plt.xlabel('Longitude')
## plt.ylabel('Latitude')
## plt.title('Points on the Map')
## plt.grid(True)
## plt.show()



## map the data:
gs_buffer.explore()







   year        gridnum_10m  species_id  B       C  east_number  north_number  \
0  2018  10mE401855N329204     2480242  1  5000.0       401855        329204   
1  2018  10mE401856N328589     2490266  1  5000.0       401856        328589   

   east_coordinate_epsg3035_meter  north_coordinate_epsg3035_meter  
0                         4018550                          3292040  
1                         4018560                          3285890  


In [3]:

import pandas as pd
import re

# geometry:
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.wkt import loads
import geodatasets

import matplotlib.pyplot as plt



##file_path = 'https://github.com/FAIRiCUBE/data-requests/blob/main/encoding-examples/datacube_nl_farmland_birds_1.csv'
file_path =r"N:\C2205_FAIRiCUBE\f02_data\d100_species_distribution\data\datacube_nl_farmland_birds_1.csv"
#file_path =r"N:\C2205_FAIRiCUBE\f02_data\d100_species_distribution\data\datacube_nl_farmland_birds_1_sub.csv"
df = pd.read_csv(file_path)
df.columns = ['year', 'gridnum_10m', 'species_id','B','C']

# Extraxt the EAST coordinate
def extract_east_number(text):
    numbers = re.findall(r'\d+', text)
    if len(numbers) >= 2:
        return int(numbers[1])  # Extract the second number
    else:
        return None

# Extraxt the NORTH coordinate
def extract_north_number(text):
    numbers = re.findall(r'\d+', text)
    if len(numbers) >= 2:
        return int(numbers[2])  # Extract the second number
    else:
        return None

# Apply the function to the text_column and create a new column
df['east_number'] = df['gridnum_10m'].apply(lambda x: extract_east_number(x))
df['north_number'] = df['gridnum_10m'].apply(lambda x: extract_north_number(x))

## get final coordinate: - for 10m reslolution:
df['east_coordinate_epsg3035_meter'] = df['east_number'] * 10
df['north_coordinate_epsg3035_meter'] = df['north_number'] * 10


#print (df)
## geometry to df:
# Create geometry column using latitude and longitude

geometry = [Point(xy) for xy in zip(df['east_coordinate_epsg3035_meter'], df['north_coordinate_epsg3035_meter'])]


# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=geometry)


## build square polygon from point: from lower left starting:

def create_square_polygon(row, side_length):
    # Define the coordinates of the square's vertices
    north_point = Point(row['geometry'].x, row['geometry'].y + side_length)
    west_point = Point(row['geometry'].x + side_length, row['geometry'].y)
    northwest_point = Point(row['geometry'].x + side_length, row['geometry'].y + side_length)

    # Create the square polygon
    square = Polygon([row['geometry'], north_point, northwest_point, west_point, row['geometry']])
    
    return square

side_length = 100  ## size of the GRIDCELL

# Apply the create_square_polygon function to each row of the GeoDataFrame
gdf['square_polygon'] = gdf.apply(create_square_polygon, args=(side_length,), axis=1)


# Convert DataFrame to GeoDataFrame
gdf_vector_grid = gpd.GeoDataFrame(gdf, geometry='square_polygon')

# Specify the projection
gdf_vector_grid.crs = "EPSG:3035" 



## map the data:
gdf_vector_grid.explore()






